# Introduction à l'apprentissage automatique: TP3 - Exercice 2 - <font color=red>CORRECTION</font>

<br>

### Reconnaissance de chiffres manuscrits

<br>

La cellule ci-dessous permet de charger 10000 images de chiffres manuscrits extraits de la célèbre base de données MNIST décrite __[ici](https://www.openml.org/d/554)__. La base originale contient 70000 images de taille 28x28 pixels, mais on restreint le nombre d'images afin de garder des temps de calcul raisonnables dans ce TD. On utilise 9000 observations comme base d'apprentissage, et on réserve 1000 observations comme base de test.

Les 28x28=784 caractéristiques sont les niveaux de gris en chaque pixel. Les caractéristiques seront normalisées à des valeurs entre 0 et 1.


In [ ]:
from sklearn import datasets, neighbors, linear_model, metrics
%matplotlib inline 

# dataset natif sklearn (ce n'est pas MNIST): (lignes suivantes à "décommenter" pour utiliser ce jeu de données)
# size_images=(8,8)
# digits = datasets.load_digits()
# X_digits = digits.data
# y_digits = digits.target

# Mnist database: (il faut quelques dizaines de secondes pour charger la base)
# les données sont décrites ici: https://www.openml.org/d/554
size_images=(28,28)
X_digits, y_digits = datasets.fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, parser='auto')
X_digits=X_digits[:10000,:]/255. # on normalise le niveau de gris 8 bits entre 0 et 1
y_digits=y_digits[:10000]

n_samples = len(X_digits)
print("nombre total d'observations (apprentissage + test): %d" % n_samples)

n_features = len(X_digits[0])
print("nombre de caractéristiques par observation: %d" % n_features)

X_train = X_digits[: 9000]
y_train = y_digits[: 9000]
X_test = X_digits[9000 :]
y_test = y_digits[9000 :]
print("nombre d'observations dans la base d'apprentissage: %d" %len(X_train))
print("nombre d'observations dans la base de test: %d" %len(X_test))


La cellule suivante définit une fonction qui permet d'afficher les 150 premières images de la base de test, ainsi que la classe véritable et la classe déterminée par l'algorithme de classification (passées en argument de la fonction). Nous nous servirons de cette fonction plus tard.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def affichage_150_images(X_test,y_test,y_pred):
    plt.figure(figsize=[15,12])   
    for n in range(150):
        plt.subplot(10,15,n+1,xticks=[],yticks=[])
        plt.imshow(np.reshape(X_test[n,:],size_images),cmap='gray_r')
        if y_pred[n]==y_test[n]:
            plt.text(0.1,0.1,str(y_pred[n])+' / '+str(y_test[n]),fontsize=6,bbox=dict(facecolor='white', alpha=1))    
        else:
            plt.text(0.1,0.1,str(y_pred[n])+' / '+str(y_test[n]),fontsize=6,bbox=dict(facecolor='red', alpha=1))    
    plt.suptitle('classe predite / classe réelle')
    plt.show();

La cellule suivante effectue une classification au plus proche voisin (`n_neighbors=1`) de la base des chiffres manuscrits extraite de MNIST, et affiche le résultat de la classification de 150 images.

__Question 1__. A quoi correspondent les temps de calcul affichés? Les erreurs vous semblent-elles qualitativement explicables? Notez le score de précision (proportion d'observations correctement classées). 

In [ ]:
# classification au plus proche voisin et affichage
knn = neighbors.KNeighborsClassifier(n_neighbors=1) 
%time knn.fit(X_train, y_train)
%time y_pred_nn = knn.predict(X_test)
print('\nKNN score: %f' % metrics.accuracy_score(y_test, y_pred_nn))
affichage_150_images(X_test,y_test,y_pred_nn)        


<font color=red>
   
Bien entendu, on ne calcule pas le score sur la base _train_... (le p.p.v. de chaque élément de _train_ est lui-même, donc on aurait un score de 1!)

Le score de classification est relativement impressionnant pour un algorithme aussi simple. En fait, MNIST est un dataset assez "facile".
    
Concernant les temps de calcul, on regarde _Wall time_: le premier concerne `fit`, le second `predict`.

`fit` prend un peu de temps mais cela reste raisonnable: cette fonction ne fait qu'organiser la base d'apprentissage pour permettre la recherche du plus proche voisin avec `predict`. _Vous avez peut-être vu les kd-trees en cours d'informatique'..._
    
`predict` prend un temps assez important (selon votre machine, une demi- à plusieurs secondes pour classifier ces 1000 observations), même si la recherche du plus proche voisin est faite intelligemment (voir la documentation: on ne fait pas une recherche exhaustive mais plutôt kd-tree, on n'en dira pas plus dans ce cours)

    
<br>
    
Avec un peu d'imagination, les erreurs s'expliquent car certains chiffres sont vraiment mal tracés et ressemblent effectivement à d'autres chiffres.

<br>
    
__Remarque__: on peut profiter de la parallélisation de la recherche du plus proche voisin avec un processeur multicoeur en passant l'argument `n_jobs=8` (par exemple, si vous avez un processeur à 8 coeurs) à `KNeighborsClassifier`. `n_jobs=-1` utilise tous les coeurs disponibles. La valeur par défaut est `n_jobs=1` dans la version courante de scikit-learn.
    
La sortie de `%time` dépend de l'OS. `Wall time` désigne le temps réellement écoulé ( _as would be seen on a wall clock_ ) pour exécuter la ligne qui suit dans le carnet Jupyter. C'est la valeur qui nous intéresse ici, et la seule visible sur tous les OS.
    
__Complément__. Pour ceux d'entre vous qui se poseraient la question sur MacOS, Linux, ou différentes versions de Windows, `%time` peut afficher: `wall time` ou `real time` (le temps écoulé sur la pendule murale - wall), `user time` ou `CPU time` qui désigne le temps total (cumulé sur tous les coeurs) consacré au calcul dans le carnet Jupyter, et `sys time` pour le temps total de calcul consacré à certains accès système comme des accès mémoire. Les deux dernières durées sont cumulées sur tous les coeurs du processeur: sur un système multi-coeurs, `wall time` est inférieur à la somme `user time` + `sys time` lorsque le programme exécuté tire parti de l'exécution en parallèle sur plusieurs coeurs (ce n'est pas le cas de toutes les fonctions Scikit-learn). `wall time` est de l'ordre de (`user time` + `sys time`)/`nb_coeurs` où `nb_coeurs` est le nombre de coeurs utilisés, si Jypyter notebook est le seul processus exécuté sur la machine (ce n'est jamais le cas, l'utilisation du CPU doit être partagée).
    

</font>

__Question 2__. Quelles sont les informations fournies par `classification_report` et `confusion_matrix` du module `metrics` ? 


In [ ]:
print(metrics.classification_report(y_test,y_pred_nn))
# cf http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html#sklearn.metrics.precision_recall_fscore_support

print(metrics.confusion_matrix(y_test,y_pred_nn))


<font color=red>
    
### Commentaires:
    
(voir exercice 1)
    
cf [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html#sklearn.metrics.precision_recall_fscore_support)
    
* precision = tp / (tp+fp)
* recall = tp / (tp+fn)   (rappel)
* f1_score: 2*precision*recall/(precision + recall)  (moyenne harmonique de precision et recall)
* support: nombre d'observations classées dans classe k

où tp = true positive, fp = false positive, fn = false negative
 
interprétation:  
* tp+fp: nombre d'observations classées dans la classe k
* tp+fn: nombre d'observations réellement dans la classe k
* precision = proportion d'observations classées dans k à raison
* recall = proportion d'observation bien classées dans k parmi tous les éléments réellement dans la classe k  
 
on veut precision et recall proches de 1
 
 
ici, on voit que 3, 4, 8 et 9 ont les moins bonnes precisions et recall (et f-score)
 
<br>

En ce qui concerne `confusion_matrix`, d'après la documentation, les vraies classes sont en lignes, les classes prédites en colonnes. Par exemple, 4 chiffres "4" ont été reconnus comme des "9"; 5 chiffres "9" ont été reconnus comme des "4".


</font>

__Question 3__. Comparez aux résultats obtenus par 

- la classification naïve bayésienne gaussienne décrite dans la [documentation scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)
    
-  à la régression logistique, décrite dans la [documentation scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
  
Comparez également les temps de calcul.

In [ ]:
from sklearn import naive_bayes

# classifieur naif:
NB = naive_bayes.GaussianNB()
%time NBfit=NB.fit(X_train, y_train)
# remarque: on accède aux paramètres estimés dans le modèle par:
# print(NB.class_prior_)
# print(NB.theta_)
# print(NB.sigma_)

%time y_pred_nb = NBfit.predict(X_test)

print('score GNB: %.3f' % metrics.accuracy_score(y_test, y_pred_nb))
    
affichage_150_images(X_test,y_test,y_pred_nb)

print(metrics.classification_report(y_test,y_pred_nb))

print(metrics.confusion_matrix(y_test,y_pred_nb))


In [ ]:
from sklearn import linear_model

LR = linear_model.LogisticRegression(max_iter=1000)  # si ConvergenceWarning, on augmente max_iter
%time LR.fit(X_train, y_train)
%time y_pred_lr=LR.predict(X_test)

print('score LR: %2f' % metrics.accuracy_score(y_test, y_pred_lr))
    
affichage_150_images(X_test,y_test,y_pred_lr)

print(metrics.classification_report(y_test,y_pred_lr))

print(metrics.confusion_matrix(y_test,y_pred_lr))

<font color=red>

Dans les deux cas (LR et GNB), les scores sont plus faibles que pour le simple classifieur au 1-p.p.v. GNB est assez faible, la "simple" régression logistique est très bonne (presque autant que 1-p.p.v). Le score faible de GNB s'explique par l'hypothèse d'indépendance conditionnelle: ce qui permet de reconnaître les chiffres est justement la corrélation entre les niveaux de gris des pixels, en particulier des pixels adjacents. 
    
De si "bons" scores pour LR et 1-p.p.v. (enfin, cela dépend de l'application visée) peuvent sembler surprenant, mais la classification de chiffres n'est en fait pas un problème difficile. De fait, les codes postaux sont lus de manière automatique depuis assez longtemps.
 
Les temps de prédiction sont bien plus faibles que pour le classifieur p.p.v.: quelques millisecondes pour classer 1000 observations par LR, ce qui rend le procédé utilisable dans un environnement industriel (lecture de codes postaux ou de chèques à la volée par exemple), contrairement à p.p.v.
    
L'apprentissage dure quelques secondes pour LR: ce n'est pas critique car c'est une opération faite une fois pour toutes.
    
Le temps de recherche rend les p.p.v. difficilement utilisables en pratique, même si dans cet exemple il fournit la meilleure précision de prédiction parmi les trois méthodes.
    
Dans le tableau de visualisation, les chiffres donnant des erreurs semblent effectivement ambigüs visuellement.
    
</font>


__Remarque__: les méthodes modernes arrivent à des précisions supérieures à 99% (sur la base MNIST entière)
Voir les "error rates" sur: http://yann.lecun.com/exdb/mnist/

